# Исследование надежности заемщиков / Borrower reliability research

The customer is the credit department of the bank.

Task: to understand whether the client’s marital status and number of children influence the fact of repaying the loan on time.

Input data from the bank - statistics on the solvency of clients.

The research results will be taken into account when constructing a credit scoring model.

Data description:

- children — number of children in the family
- days_employed — total length of service in days
- dob_years — client age in years
- education — client’s education level
- education_id — education level identifier
- family_status - marital status
- family_status_id — marital status identifier
- gender — client’s gender
- income_type — employment type
- debt — whether there was a debt to repay loans
- total_income — monthly income
- purpose — purpose of obtaining a loan

In [1]:
import pandas as pd
import warnings

warnings.filterwarnings('ignore')

try:
    data = pd.read_csv('/Users/vladamalkina/Desktop/ЯП проекты/проект 2/data_second_project.csv')
except:
    data = pd.read_csv('https://code.s3.yandex.net/datasets/data.csv')

In [2]:
data.head(20)

Unnamed: 0  children  days_employed  dob_years            education  \
0            0         1   -8437.673028         42               высшее   
1            1         1   -4024.803754         36              среднее   
2            2         0   -5623.422610         33              Среднее   
3            3         3   -4124.747207         32              среднее   
4            4         0  340266.072047         53              среднее   
5            5         0    -926.185831         27               высшее   
6            6         0   -2879.202052         43               высшее   
7            7         0    -152.779569         50              СРЕДНЕЕ   
8            8         2   -6929.865299         35               ВЫСШЕЕ   
9            9         0   -2188.756445         41              среднее   
10          10         2   -4171.483647         36               высшее   
11          11         0    -792.701887         40              среднее   
12          12         0            NaN         65              среднее   
13          13         0   -1846.641941         54  неоконченное высшее   
14          14         0   -1844.956182         56               высшее   
15          15         1    -972.364419         26              среднее   
16          16         0   -1719.934226         35              среднее   
17          17         0   -2369.999720         33               высшее   
18          18         0  400281.136913         53              среднее   
19          19         0  -10038.818549         48              СРЕДНЕЕ   

    education_id     family_status  family_status_id gender income_type  debt  \
0              0   женат / замужем                 0      F   сотрудник     0   
1              1   женат / замужем                 0      F   сотрудник     0   
2              1   женат / замужем                 0      M   сотрудник     0   
3              1   женат / замужем                 0      M   сотрудник     0   
4              1  гражданский брак                 1      F   пенсионер     0   
5              0  гражданский брак                 1      M   компаньон     0   
6              0   женат / замужем                 0      F   компаньон     0   
7              1   женат / замужем                 0      M   сотрудник     0   
8              0  гражданский брак                 1      F   сотрудник     0   
9              1   женат / замужем                 0      M   сотрудник     0   
10             0   женат / замужем                 0      M   компаньон     0   
11             1   женат / замужем                 0      F   сотрудник     0   
12             1  гражданский брак                 1      M   пенсионер     0   
13             2   женат / замужем                 0      F   сотрудник     0   
14             0  гражданский брак                 1      F   компаньон     1   
15             1   женат / замужем                 0      F   сотрудник     0   
16             1   женат / замужем                 0      F   сотрудник     0   
17             0  гражданский брак                 1      M   сотрудник     0   
18             1    вдовец / вдова                 2      F   пенсионер     0   
19             1         в разводе                 3      F   сотрудник     0   

     total_income                                 purpose  
0   253875.639453                           покупка жилья  
1   112080.014102                 приобретение автомобиля  
2   145885.952297                           покупка жилья  
3   267628.550329              дополнительное образование  
4   158616.077870                         сыграть свадьбу  
5   255763.565419                           покупка жилья  
6   240525.971920                       операции с жильем  
7   135823.934197                             образование  
8    95856.832424                   на проведение свадьбы  
9   144425.938277                 покупка жилья для семьи  
10  113943.491460                    покупка недвижимости  
11   7

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Unnamed: 0        21525 non-null  int64  
 1   children          21525 non-null  int64  
 2   days_employed     19351 non-null  float64
 3   dob_years         21525 non-null  int64  
 4   education         21525 non-null  object 
 5   education_id      21525 non-null  int64  
 6   family_status     21525 non-null  object 
 7   family_status_id  21525 non-null  int64  
 8   gender            21525 non-null  object 
 9   income_type       21525 non-null  object 
 10  debt              21525 non-null  int64  
 11  total_income      19351 non-null  float64
 12  purpose           21525 non-null  object 
dtypes: float64(2), int64(6), object(5)
memory usage: 2.1+ MB


## Предобработка данных / Data preprocessing

### Удаление пропусков / Removing gaps

Подсчет количества пропусков: / Counting the number of passes:

In [4]:
data.isna().sum()

Unnamed: 0             0
children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

В двух столбцах есть пропущенные значения: days_employed и total_income. 
total_income зависит от income_type, заполним пропуски в этом столбце медианным значением по каждому типу из столбца income_type:

Two columns have missing values: days_employed and total_income.
total_income depends on income_type, fill in the gaps in this column with the median value for each type from the income_type column:

In [5]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['total_income'].isna()), 'total_income'] = \
    data.loc[(data['income_type'] == t), 'total_income'].median()

### Обработка аномальных значений / Handling anomalous values

В данных встретились аномалии - отрицательное количество дней трудового стажа в столбце days_employed. 
Обработаем значения в этом столбце: заменим все отрицательные значения положительными с помощью метода abs():

There were anomalies in the data - a negative number of days of work experience in the days_employed column.
Let's process the values ​​in this column: replace all negative values ​​with positive ones using the abs() method:

In [6]:
data['days_employed'] = data['days_employed'].abs()

Для каждого типа занятости выведем медианное значение трудового стажа days_employed в днях:

For each type of employment, we display the median value of length of service days_employed in days:

In [7]:
data.groupby('income_type')['days_employed'].agg('median')

income_type
безработный        366413.652744
в декрете            3296.759962
госслужащий          2689.368353
компаньон            1547.382223
пенсионер          365213.306266
предприниматель       520.848083
сотрудник            1574.202821
студент               578.751554
Name: days_employed, dtype: float64

У двух типов (безработные и пенсионеры) получатся аномально большие значения. 
Исправить такие значения сложно, поэтому оставим их как есть. 
Этот столбец не понадобится для исследования.

Two types (unemployed and pensioners) will have abnormally large values.
It is difficult to correct such values, so we will leave them as they are.
This column will not be needed for the study.

In [8]:
data['children'].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5])

В столбце children есть два аномальных значения. Удалим строки, в которых встречаются такие аномальные значения из датафрейма:

There are two anomalous values ​​in the children column. Let's remove lines containing such anomalous values ​​from the dataframe:

In [9]:
data = data[(data['children'] != -1) & (data['children'] != 20)]

Выведем перечень уникальных значений столбца children, чтобы убедиться, что аномалии удалены:

Let's list the unique values ​​of the children column to make sure that anomalies have been removed:

In [10]:
data['children'].unique()

array([1, 0, 3, 2, 4, 5])

Заполним пропуски в столбце days_employed медианными значениями по каждому типу занятости income_type:

Let's fill in the gaps in the days_employed column with the median values ​​for each income_type employment type:

In [11]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['days_employed'].isna()), 'days_employed'] = \
    data.loc[(data['income_type'] == t), 'days_employed'].median()

Проверим количество пропусков:

Let's check the number of passes:

In [12]:
data.isna().sum()

Unnamed: 0          0
children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

Заменим вещественный тип данных в столбце total_income на целочисленный с помощью метода astype():

Let's replace the real data type in the total_income column with an integer using the astype() method:

In [13]:
data['total_income'] = data['total_income'].astype(int)

### Обработка дубликатов / Handling duplicates

Обработаем неявные дубликаты в столбце education. 
В этом столбце есть одни и те же значения, но записанные по-разному: с использованием заглавных и строчных букв. Приведем их к нижнему регистру:

Let's handle implicit duplicates in the education column.
This column has the same values, but written differently, using uppercase and lowercase letters. Let's convert them to lower case:

In [14]:
data['education'] = data['education'].str.lower()

Выведем на экран количество строк-дубликатов в данных:

Let's display the number of duplicate rows in the data:

In [15]:
data.duplicated().sum()

0

In [16]:
data = data.drop_duplicates()

### Категоризация данных / Data categorization

Создадим диапозон доходов, на его основании создадим в датафрейме столбец total_income_category с категориями:

- 0–30000 — 'E';
- 30001–50000 — 'D';
- 50001–200000 — 'C';
- 200001–1000000 — 'B';
- 1000001 и выше — 'A'.

Let's create an income range, based on it, create a total_income_category column with categories in the dataframe:

- 0–30000 — 'E';
- 30001–50000 - 'D';
- 50001–200000 - 'C';
- 200001–1000000 - 'B';
- 1000001 and above - 'A'.

In [17]:
def categorize_income(income):
    try:
        if 0 <= income <= 30000:
            return 'E'
        elif 30001 <= income <= 50000:
            return 'D'
        elif 50001 <= income <= 200000:
            return 'C'
        elif 200001 <= income <= 1000000:
            return 'B'
        elif income >= 1000001:
            return 'A'
    except:
        pass

In [18]:
data['total_income_category'] = data['total_income'].apply(categorize_income)

In [19]:
data['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

Создадим функцию, которая на основании данных из столбца purpose сформирует новый столбец purpose_category, в который войдут следующие категории:

- 'операции с автомобилем',
- 'операции с недвижимостью',
- 'проведение свадьбы',
- 'получение образования'.

Let's create a function that, based on the data from the purpose column, will form a new purpose_category column, which will include the following categories:

- 'car operations',
- 'real estate transactions',
- 'carrying out a wedding',
- 'getting an education'.

In [20]:
def categorize_purpose(row):
    try:
        if 'автом' in row:
            return 'операции с автомобилем'
        elif 'жил' in row or 'недвиж' in row:
            return 'операции с недвижимостью'
        elif 'свад' in row:
            return 'проведение свадьбы'
        elif 'образов' in row:
            return 'получение образования'
    except:
        return 'нет категории'

In [21]:
data['purpose_category'] = data['purpose'].apply(categorize_purpose)

### Шаг 3. Исследуем данные / Step 3. Explore the data

#### 3.1 Проверим, есть ли зависимость между количеством детей и возвратом кредита в срок: / Let's check whether there is a relationship between the number of children and repayment of the loan on time:

In [22]:
import warnings

warnings.filterwarnings('ignore')
grouped_no_children = data.loc[data['children']==0,'debt'].agg(['sum','count','mean'])
grouped_has_children = data.loc[data['children']!=0, 'debt'].agg(['sum','count','mean'])
print(grouped_no_children.append(grouped_has_children))

sum       1063.000000
count    14149.000000
mean         0.075129
sum        669.000000
count     7253.000000
mean         0.092238
Name: debt, dtype: float64


**Вывод:** существует зависимость между наличием детей у клиентов и задолженностями. Клиенты-должники с детьми составляют большую долю от заемщиков, чем клиенты без детей.

**Conclusion:** there is a relationship between the presence of children of clients and debts. Debtor clients with children make up a larger share of borrowers than clients without children.

#### 3.2 Проверим, есть ли зависимость между семейным положением и возвратом кредита в срок: / Let's check whether there is a relationship between marital status and repayment of the loan on time:

In [23]:
print(data.groupby('family_status')['debt'].agg(['sum','count','mean']))

                       sum  count      mean
family_status                              
Не женат / не замужем  273   2799  0.097535
в разводе               84   1189  0.070648
вдовец / вдова          63    952  0.066176
гражданский брак       385   4160  0.092548
женат / замужем        927  12302  0.075354


**Вывод:** существует зависимость между семейным положением и возвратом кредита в срок. Наибольшее число долгов у клиентов из категорий не женат/не замужем и гражданский брак, меньше всего долгов у клиентов из категории вдовец/вдова.  

**Conclusion:** there is a relationship between marital status and repayment of the loan on time. Clients from the unmarried and civil marriage categories have the largest number of debts, clients from the widower/widow category have the least debts.

#### 3.3 Проверим, есть ли зависимость между уровнем дохода и возвратом кредита в срок: / Let's check whether there is a relationship between income level and loan repayment on time:

In [24]:
print(data.groupby('total_income_category')['debt'].agg(['sum','count','mean']))

                        sum  count      mean
total_income_category                       
A                         2     25  0.080000
B                       354   5014  0.070602
C                      1353  15992  0.084605
D                        21    349  0.060172
E                         2     22  0.090909


**Вывод:** зависимость между уровнем дохода и возвратом кредита в срок присутствует. Больше всего задолженностей у клиентов, относящихся к категории Е. Меньше всего долгов у клиентов из категории D. Клиенты-должники из категорий А, B и С составляют примерно равные доли от общего числа заемщиков.

**Conclusion:** there is a relationship between income level and loan repayment on time. Clients belonging to category E have the most debts. Clients from category D have the least debts. Clients-debtors from categories A, B and C make up approximately equal shares of the total number of borrowers.

#### 3.4 Проверим, как разные цели кредита влияют на его возврат в срок: / Let's check how different purposes of a loan affect its repayment on time:

In [25]:
print(data.groupby('purpose_category')['debt'].agg(['sum','count','mean']))

                          sum  count      mean
purpose_category                              
операции с автомобилем    400   4288  0.093284
операции с недвижимостью  780  10780  0.072356
получение образования     369   3997  0.092319
проведение свадьбы        183   2337  0.078306


**Вывод:** больше  задолженностей имеют клиенты, которые проводили операции с автомобилем или брали кредит на получение образования.
Меньше задолженностей у клиентов, которые берут кредит на проведение свадьбы или операции с недвижимостью.

**Conclusion:** clients who carried out transactions with a car or took out a loan for education have more debt.
Clients who take out a loan for a wedding or real estate transaction have less debt.

В ходе исследования было проверено и установлено: семейное положение клиента влияет на факт погашения кредита в срок. Наибольшее число долгов у клиентов из категорий не женат/не замужем и гражданский брак, меньше всего долгов у клиентов из категории вдовец/вдова.  

Существует зависимость между наличием детей у клиентов и задолженностями. Клиенты-должники с детьми составляют большую долю от заемщиков, чем клиенты без детей.

Также присутствует зависимость между уровнем дохода и возвратом кредита в срок. Больше всего задолженностей у клиентов, относящихся к категории Е. Меньше всего долгов у клиентов из категории D. Клиенты-должники из категорий А, B и С составляют примерно равные доли от общего числа заемщиков.

Более того, на факт погашения кредита в срок влияет цель получения кредита:

- больше  задолженностей имеют клиенты, которые проводили операции с автомобилем или брали кредит на получение образования;
- меньше задолженностей у клиентов, которые берут кредит на проведение свадьбы или операции с недвижимостью.


During the study, it was verified and established that the client’s marital status affects the fact of repaying the loan on time. Clients from the unmarried and civil marriage categories have the largest number of debts, clients from the widower/widow category have the least debts.

There is a relationship between the presence of children of clients and debt. Debtor clients with children make up a larger share of borrowers than clients without children.

There is also a relationship between income level and loan repayment on time. Clients belonging to category E have the most debts. Clients from category D have the least debts. Clients-debtors from categories A, B and C make up approximately equal shares of the total number of borrowers.

Moreover, the fact of repaying the loan on time is influenced by the purpose of obtaining the loan:

- clients who carried out transactions with a car or took out a loan for education have more debts;
- less debt for clients who take out a loan for a wedding or real estate transaction.